In [1]:
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import numpy as np
import uproot
import os
import glob
import awkward as ak
from scipy.stats import norm

In [24]:
setup_path ="/home/haue/repositories/Masters/setup/" 
data_path = "/home/haue/repositories/Masters/data/singleprotons/Al/inclbetacut/"
twop_path = "/home/haue/repositories/Masters/data/twoproton/Al/"
Al_files = glob.glob(os.path.join(data_path, "*mlio.root"))
twop_files = glob.glob(os.path.join(twop_path, "*mlio.root"))


frac = 0.9542011788469517 # (Na21/(p+Na21)) amount of energy the daughter nucleus gets of the total energy released in the decay
frac_err = 1.032071836027589e-10 # uncertainty of the fraction
psep = 5.50410 # proton separation energy of 22Mg
psep_err = 0.00016 # error of proton separation energy of 22Mg
scale = 3.326542500626603 # difference in solid angle coverage of two proton events up to 4pi
sac4 = 6.795917343263473 # solid angle coverage of id3

In [3]:
E2_id3 = np.array([])
for batch in uproot.iterate(Al_files, expressions=["E"],
                            cut="(id==3)"):
    E2_id3=np.append(E2_id3, ak.flatten(batch.E))
E2p_id3 = np.array([])
for batch in uproot.iterate(twop_files, expressions=["E"],
                            cut="(id==3)"):
    E2p_id3=np.append(E2p_id3, ak.flatten(batch.E))
    
E2_id2 = np.array([])
for batch in uproot.iterate(Al_files, expressions=["E"],
                            cut="(id==2)"):
    E2_id2=np.append(E2_id2, ak.flatten(batch.E))
E2p_id2 = np.array([])
for batch in uproot.iterate(twop_files, expressions=["E"],
                            cut="(id==2)"):
    E2p_id2=np.append(E2p_id2, ak.flatten(batch.E))




In [36]:
y1p_id3, _ = np.histogram(E2_id3, bins=np.arange(0,9000+10,10))
y2p_id3, _ = np.histogram(E2p_id3, bins=np.arange(0,9000+10,10))
y1p_id2, _ = np.histogram(E2_id2, bins=np.arange(0,1000+10,10)) #to find the missing counts from id3 spectrum
y2p_id2, _ = np.histogram(E2p_id2, bins=np.arange(0,1000+10,10)) # to find the missing counts from id3 spectrum

counts_1p_id3 = np.sum(y1p_id3); counts_1p_id3_err = np.sqrt(np.sum(y1p_id3))
counts_2p_id3 = np.sum(y2p_id3); counts_2p_id3_err = np.sqrt(np.sum(y2p_id3))
counts_1p_id2 = np.sum(y1p_id2); counts_1p_id2_err = np.sqrt(np.sum(y1p_id2))
counts_2p_id2 = np.sum(y2p_id2); counts_2p_id2_err = np.sqrt(np.sum(y2p_id2))
relative_diff = 11122/9083 # relative peaksizes of 1.245 peak of id3/id2 186, 290
#relative_diff_err = 0.05
relative_diff_err = np.sqrt((186 / 9083)**2 + (11122 * 290 / 9083**2)**2+0.05**2)
#print(relative_diff_err)
relative_solidangle = 100/sac4

tot_1p_counts = (counts_1p_id3-counts_2p_id3*scale + counts_1p_id2*relative_diff-counts_2p_id2*scale*relative_diff ) * relative_solidangle # total 1p counts in id3 where the region below 1MeV has been estimated from id2
tot_1p_counts_err = np.sqrt((counts_1p_id3_err)**2 + (counts_2p_id3_err * scale)**2 + (counts_1p_id2_err * relative_diff)**2 + (counts_2p_id2_err * scale * relative_diff)**2 + (counts_1p_id2 * relative_diff_err)**2 + (counts_2p_id2 * scale * relative_diff_err)**2) * relative_solidangle

tot_2p_counts = ((counts_2p_id3*scale+counts_2p_id2*scale*relative_diff) * relative_solidangle) /2 # divided by 2 because 1 2p event is two protons
tot_2p_counts_err = 0.5 * relative_solidangle * np.sqrt((counts_2p_id3_err * scale)**2 + (counts_2p_id2_err * scale * relative_diff)**2 + (counts_2p_id2 * scale * relative_diff_err)**2)

c1985 = 429866
c1985_err = 58849
tot_g_counts = 532820# weighted mean calculated from counts in 2061 peak in G1 & G2
tot_g_counts_err = 31219

tot = tot_1p_counts+tot_2p_counts+tot_g_counts
tot_err = np.sqrt(tot_1p_counts_err**2+tot_2p_counts_err**2+tot_g_counts_err**2)
print(f"Total number of events recorded(not including alphas yet): {tot:.0f} +/- {tot_err:.2f}")
print(f"Branching ratio 1p: {tot_1p_counts/tot*100:.2f} +/- {100 * np.sqrt((tot_1p_counts_err / tot)**2 + (tot_1p_counts * tot_err / tot**2)**2):.2f}")
print(f"Branching ratio 2p: {tot_2p_counts/tot*100:.2f} +/- {100 * np.sqrt((tot_2p_counts_err / tot)**2 + (tot_2p_counts * tot_err / tot**2)**2):.2f}")
print(f"Branching ratio g: {tot_g_counts/tot*100:.2f} +/- {100 * np.sqrt((tot_g_counts_err / tot)**2 + (tot_g_counts * tot_err / tot**2)**2):.2f}")
print()
print(f"Total proton events  {tot-tot_g_counts:.0f}")


Total number of events recorded(not including alphas yet): 1148683 +/- 32779.44
Branching ratio 1p: 50.04 +/- 1.67
Branching ratio 2p: 3.58 +/- 0.14
Branching ratio g: 46.39 +/- 3.02

Total proton events  615863


In [41]:
def rel(totc,totc_e,c, c_e):
    br = (c/totc) *100
    br_e = br*np.sqrt((c_e/c)**2+(totc_e/totc)**2)
    return br,br_e

def weighted_mean(values, errors):
    values = np.asarray(values)
    errors = np.asarray(errors)
    weights = 1.0 / (errors**2)
    mean = np.sum(weights * values) / np.sum(weights)
    err_mean = np.sqrt(1.0 / np.sum(weights))
    return mean, err_mean

print(rel(tot,tot_err,11122*relative_solidangle,186*relative_solidangle))

(14.24737223180405, 0.4712443527961572)


In [ ]:

ps3 = [1249,1167,1311,2486,2396,2563,5456,3880,1670,6517,7843]
cs3 = [11129,306,596,3741,66,130,144,363,496,100,68]
ce3 = [630,33,47,371,8,23,11,39,321,20,8]

cs2 = [9084,118,179,210,352,   1402,256,3242,1919,358, 223,72]
ce2 = [973,21,10,32,43,          55,27,103,288,45,26,14]
ps2 = [1249,1167,1311,5456,3880,  996,775,693,452,1670,6522,7844]

cs1 = [8604,161,116,225,335,1186,260,2237,332, 205, 86]
ce1 = [922,17,6,29,40,66,30,55, 42, 34,16]
ps1 = [1249,1167,1311,5456,3880,996,775,693,1670, 6524, 7847]




"""
for c,p,e in zip(cs1,ps1,ce1):
    print(p, rel(cs1[0], ce1[0], c,e))

    
print("="*60)
for c,p,e in zip(cs2,ps2,ce2):
    print(p, rel(cs2[0], ce2[0], c,e))


print("="*60)
for c,p,e in zip(cs3,ps3,ce3):
    print(p, rel(cs3[0], ce3[0], c,e))
print("="*60)
print("="*60)

"""
absscale, absscale_err = rel(tot,tot_err,cs3[0]*relative_solidangle,ce3[0]*relative_solidangle)
absscale /= 100
absscale_err /= 100
p1249 = weighted_mean([100,100,100],[15.154636268107783,15.147840116567826,8.005701718888039])
p1167 = weighted_mean([1.8712226871222686,1.2989872302950243,2.749573187168658],[0.28150809457245957,0.26981683472575363,0.3348919783281118])
p1311 = weighted_mean([1.3482101348210134,1.9704975781594012,5.355377841674904], [0.16042305013423053,0.2380460105115358,0.5198666502826514])

p5456 = weighted_mean([2.615062761506276,2.3117569352708056,1.2939167939617218], [0.3590475232387749,0.4305879712403336,0.12302301591640319])
p3880 = weighted_mean([3.893537889353789,3.8749449581682076,3.2617485847785064], [0.4894032231917866,0.6295527147416979,0.3961042877995525])
p996 = weighted_mean([13.784286378428638, 15.433729634522237], [1.6644197921713146,1.760515748159705]) # id1,id2
p775 = weighted_mean([3.0218503021850305, 2.8181417877586967], [0.475850369740362, 0.42362689649911534]) # id1,id2
p693 = weighted_mean([25.99953509995351, 35.68912373403787], [2.858488495145724, 3.98732627245805]) # id1,id2
p452 = weighted_mean(21.125055041831793, 3.895056234281918) #id2
p2486 = weighted_mean(33.614880043130555, 3.838507067582582) #id3
p2396 =weighted_mean(0.5930451972324557, 0.07933726050380133) #id3
p2563 =weighted_mean(1.1681193278821098, 0.21698846463602584) #id3
p1670 = weighted_mean([3.8586703858670384,3.9409951563188024,4.456824512534819], [0.6397359461924,0.65083624940473,2.8953693647207346])
p6417 =weighted_mean([2.382612738261274,2.454865697930427,0.8985533291400845], [0.4704715225798862,0.388664480565854,0.1867706653714041]) #id3
p7844 =weighted_mean([0.9995350999535101,0.7926023778071334,0.6110162638152574], [0.214601034445497,0.1759532560129451,0.07977306625344734]) #id3


peaks = [p1249,p1167,p1311,p5456,p3880,p996,p775,p693,p452,p2486,p2396,p2563,p1670,p6417,p7844]
peakvals = [1249,1167,1311,5456,3880,996,775,693,452,2486,2396,2563,1673,6517,7844]
brs = []
for (mean, err),p in zip(peaks,peakvals):
    abs_mean = mean * absscale
    abs_err = abs_mean * np.sqrt((err / mean)**2 + (absscale_err / absscale)**2)
    brs.append(abs_mean)
    print(f"E={p} - {abs_mean:.2f} +/- {abs_err:.2f}")
print(f"Total branching ratio accounted for: {np.sum(brs):.2f} %")
print(f"Total 1-p branching ratio {tot_1p_counts/tot*100:.2f} % +/- {100 * np.sqrt((tot_1p_counts_err / tot)**2 + (tot_1p_counts * tot_err / tot**2)**2):.2f} %")
print(f"Missing branching ratio: {tot_1p_counts/tot*100-np.sum(brs):.2f} %")
#"""


E=1249 - 14.26 +/- 1.29
E=1167 - 0.27 +/- 0.03
E=1311 - 0.25 +/- 0.02
E=5456 - 0.21 +/- 0.02
E=3880 - 0.51 +/- 0.05
E=996 - 2.08 +/- 0.22
E=775 - 0.41 +/- 0.05
E=693 - 4.18 +/- 0.42
E=452 - 3.01 +/- 0.59
E=2486 - 4.79 +/- 0.63
E=2396 - 0.08 +/- 0.01
E=2563 - 0.17 +/- 0.03
E=1673 - 0.56 +/- 0.07
E=6517 - 0.19 +/- 0.03
E=7844 - 0.10 +/- 0.01
Total branching ratio accounted for: 31.06 %
Total 1-p branching ratio 50.04 % +/- 1.67 %
Missing branching ratio: 18.97 %
17139
